In [2]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import empty, nan, savetxt
from datetime import datetime
import random

from source.spinsystem import SpinSystem
from source.utils import read_config_file, reconstruct_grid, visualize_grid, plot_array_list
from scipy.special import softmax

In [8]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

def simulate_spin_dynamics(L=50, beta=1.0, alpha=4.0, gamma=1.0, J=1.0, steps=200):
    S = np.random.choice([-1, 0, 1], size=(L, L))
    C = np.random.choice([-1, 1], size=(L, L))
    T = np.random.choice([-1, 1], size=(L, L))
    
    snapshots = []

    def get_neighbors(i, j):
        return [((i-1) % L, j), ((i+1) % L, j), (i, (j-1) % L), (i, (j+1) % L)]

    def local_field(i, j):
        return sum(J * S[x, y] for x, y in get_neighbors(i, j))

    def magnetization():
        return np.mean(S)

    def polarization():
        return np.mean(S**2)

    def energy_levels(h_loc, M, P, Cij, Tij):
        levels = []
        for s in [-1, 0, 1]:
            H = -s * h_loc + alpha * Cij * s * M + gamma * Tij * s**2 * P
            levels.append(-beta * H)
        return softmax(levels)

    for t in range(steps):
        M = magnetization()
        P = polarization()

        for _ in range(L * L):
            i, j = np.random.randint(0, L), np.random.randint(0, L)
            h = local_field(i, j)
            probs = energy_levels(h, M, P, C[i, j], T[i, j])
            S[i, j] = np.random.choice([-1, 0, 1], p=probs)

            global_alignment = alpha * S[i, j] * C[i, j] * np.sum(S)
            if global_alignment < 0:
                C[i, j] *= -1

        #if t % 10 == 0:
        snapshots.append(S.copy())

    return snapshots

snapshots = simulate_spin_dynamics()
plot_array_list(snapshots, max_cols=10, timesteps=[i for i in range(len(snapshots))])